# Limpieza y transformación de datos

Este notebook aplica la limpieza y transformación necesarias al dataset de accidentes viales para estandarizar su estructura y facilitar el análisis posterior.

## Objetivo del notebook

Estandarizar tipos de datos en el dataset:
- Fechas → `datetime`
- Horas → `datetime.time`
- Coordenadas → `float`
- Texto categórico → `str` en minúsculas
- Booleanos → `bool`

Esto permite un análisis más confiable y evita errores en las siguientes etapas del proyecto.


## Tipos de datos en Pandas utilizados

| Tipo de dato        | Descripción                                                  |
|---------------------|--------------------------------------------------------------|
| `object`            | Tipo genérico, usualmente texto (strings).                   |
| `int64`             | Números enteros (sin decimales).                             |
| `float64`           | Números decimales (punto flotante).                          |
| `datetime64[ns]`    | Fecha y hora (datetime).                                     |
| `bool`              | Valores lógicos: `True` o `False`.                           |
| `category`          | Texto optimizado para variables categóricas repetitivas.     |

---

In [ ]:
import pandas as pd

# Cargamos el dataset original desde la carpeta /data
df = pd.read_csv('../data/accidentes_cdmx.csv')

# Mostramos resumen de todas las columnas y sus tipos de datos
print("\nTipos de datos por columna:\n")
print(df.dtypes)

# Mostramos valores únicos para columnas tipo 'object' para revisar si requieren limpieza/conversión
print("\nColumnas tipo 'object' con ejemplos de valores:\n")
for col in df.select_dtypes(include='object').columns:
    print(f"{col}:")
    print(df[col].dropna().unique()[:5])
    print("----")


In [ ]:

# Conversión de columnas que requieren cambio de tipo

# Conversión de fechas
# Convierte 'fecha_evento' a tipo datetime usando el formato día/mes/año
df['fecha_evento'] = pd.to_datetime(df['fecha_evento'], format='%d/%m/%Y', errors='coerce')

# Convierte 'fecha_captura' a datetime con el mismo formato
df['fecha_captura'] = pd.to_datetime(df['fecha_captura'], format='%d/%m/%Y', errors='coerce')

# Limpieza y conversión de hora
# Asegura que la columna 'hora_evento' sea texto limpio (sin espacios ni valores 'nan')
df['hora_evento'] = df['hora_evento'].astype(str).str.strip().replace('nan', pd.NA)

# Convierte 'hora_evento' a tipo datetime solo con hora, usando el formato 24h
df['hora_evento'] = pd.to_datetime(df['hora_evento'], format='%H:%M', errors='coerce').dt.time

# Conversión de coordenadas
# Convierte 'latitud' a tipo numérico (float), forzando errores como NaN
df['latitud'] = pd.to_numeric(df['latitud'], errors='coerce')

# Convierte 'longitud' a tipo numérico también, para evitar errores futuros
df['longitud'] = pd.to_numeric(df['longitud'], errors='coerce')  

# Conversión de columna booleana 
# Si existe la columna 'trasladado_lesionados':
if 'trasladado_lesionados' in df.columns:
    # Normaliza el texto: elimina espacios y pasa todo a minúsculas
    df['trasladado_lesionados'] = df['trasladado_lesionados'].astype(str).str.strip().str.lower()

    # Reemplaza los valores 'si' y 'no' por booleanos y convierte la columna a tipo bool
    df['trasladado_lesionados'] = df['trasladado_lesionados'].replace({'si': True, 'no': False}).astype('bool')

# Estandarización de textos en matrícula médica 
# Si existe la columna 'matricula_unidad_medica':
if 'matricula_unidad_medica' in df.columns:
    # Convierte a string, elimina espacios y pasa todo a mayúsculas
    df['matricula_unidad_medica'] = df['matricula_unidad_medica'].astype(str).str.strip().str.upper()

# Limpieza de columnas categóricas de texto
# Lista de columnas que deben estandarizarse como texto uniforme (en minúsculas, sin espacios)
columnas_texto = [
    'tipo_evento', 'folio', 'punto_1', 'punto_2', 'colonia', 'alcaldia', 'sector',
    'unidad_a_cargo', 'tipo_de_interseccion', 'clasificacion_de_la_vialidad',
    'sentido_de_circulacion', 'dia', 'prioridad', 'origen', 'unidad_medica_de_apoyo'
]

# Para cada columna en la lista:
for col in columnas_texto:
    if col in df.columns:
        # Asegura que sea texto limpio (sin espacios, en minúsculas)
        df[col] = df[col].astype(str).str.strip().str.lower()





In [ ]:
# Mostramos resumen de todas las columnas y sus tipos de datos
print("\nTipos de datos por columna:\n")
print(df.dtypes)

# Mostramos valores únicos para columnas tipo 'object' para revisar si requieren limpieza/conversión
print("\nColumnas tipo 'object' con ejemplos de valores:\n")
for col in df.select_dtypes(include='object').columns:
    print(f"{col}:")
    print(df[col].dropna().unique()[:5])
    print("----")